#custom data 훈련

#설치

In [ ]:
!git clone --recurse-submodules https://github.com/jks5177/Yolov5_DeepSort_Pytorch.git

%cd Yolov5_DeepSort_Pytorch
%pip install -qr requirements.txt

import torch
from IPython.display import Image, clear_output

Cloning into 'Yolov5_DeepSort_Pytorch'...
remote: Enumerating objects: 898, done.
remote: Total 898 (delta 0), reused 0 (delta 0), pack-reused 898
Receiving objects: 100% (898/898), 70.82 MiB | 33.10 MiB/s, done.
Resolving deltas: 100% (414/414), done.
Submodule 'yolov5' (https://github.com/ultralytics/yolov5.git) registered for path 'yolov5'
Cloning into '/content/Yolov5_DeepSort_Pytorch/Yolov5_DeepSort_Pytorch/yolov5'...
remote: Enumerating objects: 12327, done.        
remote: Counting objects: 100% (2/2), done.        
remote: Compressing objects: 100% (2/2), done.        
remote: Total 12327 (delta 0), reused 1 (delta 0), pack-reused 12325        
Receiving objects: 100% (12327/12327), 11.53 MiB | 26.94 MiB/s, done.
Resolving deltas: 100% (8573/8573), done.
Submodule path 'yolov5': checked out 'b8be76f915207ef0759bfb0f1c0707c79877b763'
/content/Yolov5_DeepSort_Pytorch/Yolov5_DeepSort_Pytorch


In [ ]:
!pip install -qr ./yolov5/requirements.txt

In [ ]:
pwd

'/content/Yolov5_DeepSort_Pytorch/Yolov5_DeepSort_Pytorch'

In [ ]:
mkdir Road_Sign_Dataset

#데이터 업로드

데이터 직접 업로드하기 

cow_data_custom.zip : /content/Yolov5_DeepSort_Pytorch/Road_Sign_Dataset/cow_data_custom.zip

In [ ]:
cd /content/Yolov5_DeepSort_Pytorch/Road_Sign_Dataset

/content/Yolov5_DeepSort_Pytorch/Road_Sign_Dataset


In [ ]:
!unzip cow_data_custom.zip

unzip:  cannot find or open cow_data_custom.zip, cow_data_custom.zip.zip or cow_data_custom.zip.ZIP.


In [ ]:
cd /content/Yolov5_DeepSort_Pytorch

/content/Yolov5_DeepSort_Pytorch


In [ ]:
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

random.seed(108)

#모델 훈련

1. 아래 파일 옆에 적어논 파일경로에 꼭!! 업로드하기

road_sign_data.yaml : /content/Yolov5_DeepSort_Pytorch/yolov5/data/road_sign_data.yaml

2. 아래 경로의파일 라인20에
/content/Yolov5_DeepSort_Pytorch/yolov5/models/common.p


```
import warnings
class SPPF(nn.Module):
    # Spatial Pyramid Pooling - Fast (SPPF) layer for YOLOv5 by Glenn Jocher
    def __init__(self, c1, c2, k=5):  # equivalent to SPP(k=(5, 9, 13))
        super().__init__()
        c_ = c1 // 2  # hidden channels
        self.cv1 = Conv(c1, c_, 1, 1)
        self.cv2 = Conv(c_ * 4, c2, 1, 1)
        self.m = nn.MaxPool2d(kernel_size=k, stride=1, padding=k // 2)
 
    def forward(self, x):
        x = self.cv1(x)
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')  # suppress torch 1.9.0 max_pool2d() warning
            y1 = self.m(x)
            y2 = self.m(y1)
            return self.cv2(torch.cat([x, y1, y2, self.m(y2)], 1))
```



이 코드 붙혀넣고 저장



In [ ]:
pwd

'/content/Yolov5_DeepSort_Pytorch'

In [ ]:
!python yolov5/train.py --img 416 --cfg yolov5s.yaml --hyp hyp.scratch.yaml --batch 32 --epochs 101 --data road_sign_data.yaml --weights ' ' --name yolo_road_det --workers 24 #yolov5s.pt

github: up to date with https://github.com/jks5177/Yolov5_DeepSort_Pytorch ✅
YOLOv5 🚀 v5.0-69-gb8be76f torch 1.10.0+cu111 CPU

Namespace(adam=False, artifact_alias='latest', batch_size=32, bbox_interval=-1, bucket='', cache_images=False, cfg='./yolov5/models/yolov5s.yaml', data='./yolov5/data/road_sign_data.yaml', device='', entity=None, epochs=101, evolve=False, exist_ok=False, global_rank=-1, hyp='./yolov5/data/hyp.scratch.yaml', image_weights=False, img_size=[416, 416], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='yolo_road_det', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/yolo_road_det2', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=32, upload_dataset=False, weights=' ', workers=24, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_de

3. /content/Yolov5_DeepSort_Pytorch/track.py의 라인 268 아래코드로 수정하기-수정해야할코드가 위에 훈련때마다 좀 바뀜



```
parser.add_argument('--yolo_weights', type=str, default='/content/Yolov5_DeepSort_Pytorch/runs/train/yolo_road_det2/weights/best.pt', help='model.pt path')
```



#예측

In [ ]:
cd /content/Yolov5_DeepSort_Pytorch

/content/Yolov5_DeepSort_Pytorch


In [ ]:
#결과 동영상 확인용
#마지막에 yolo_road_det2 수정해야할지말아야할지 돌릴때마다 확인해야함
#--weights 도 매번 수정해야함
!python yolov5/detect.py --source /content/Yolov5_DeepSort_Pytorch/sample_video.mp4 --weights /content/Yolov5_DeepSort_Pytorch/runs/train/yolo_road_det2/weights/best.pt --conf 0.25 --name yolo_road_det7

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, hide_conf=False, hide_labels=False, img_size=640, iou_thres=0.45, line_thickness=3, name='yolo_road_det7', nosave=False, project='runs/detect', save_conf=False, save_crop=False, save_txt=False, source='/content/Yolov5_DeepSort_Pytorch/sample_video.mp4', update=False, view_img=False, weights=['/content/Yolov5_DeepSort_Pytorch/runs/train/yolo_road_det2/weights/best.pt'])
YOLOv5 🚀 v5.0-69-gb8be76f torch 1.10.0+cu111 CPU

Fusing layers... 
/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 224 layers, 7053910 parameters, 0 gradients, 16.3 GFLOPS
video 1/1 (1/441) /content/Yolov5_DeepSort_Pytorch/sample_video.mp4: 384x

In [ ]:
!python track.py --source sample_video.mp4 --save-vid

/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
video 1/1 (1/441) /content/Yolov5_DeepSort_Pytorch/sample_video.mp4: 384x640 Done. (0.325s)
video 1/1 (2/441) /content/Yolov5_DeepSort_Pytorch/sample_video.mp4: video 1/1 (3/441) /content/Yolov5_DeepSort_Pytorch/sample_video.mp4: video 1/1 (4/441) /content/Yolov5_DeepSort_Pytorch/sample_video.mp4: video 1/1 (5/441) /content/Yolov5_DeepSort_Pytorch/sample_video.mp4: video 1/1 (6/441) /content/Yolov5_DeepSort_Pytorch/sample_video.mp4: video 1/1 (7/441) /content/Yolov5_DeepSort_Pytorch/sample_video.mp4: video 1/1 (8/441) /content/Yolov5_DeepSort_Pytorch/sample_video.mp4: video 1/1 (9/441) /content/Yolov5_DeepSort_Pytorch/sample_video.mp4: video 1/1 (10/441) /content/Yolov5_DeepSort_